<a href="https://colab.research.google.com/github/diego-iac/Database_Oncogeni/blob/main/download_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# librerie da installare
# !pip install bioservices
# !pip install elementpath

In [ ]:
import pandas as pd
import numpy as np
#from bioservices import *
#%pylab inline --no-import-all
#from bioservices.uniprot import UniProt
import os
from io import StringIO
from Bio import ExPASy
from Bio.ExPASy import ScanProsite
from Bio.ExPASy import Prodoc
from Bio import SeqIO
from Bio.ExPASy import Prosite
import xml.etree.ElementTree as ET


In [ ]:
pd.set_option('display.max_columns', None)


In [ ]:
# current_directory = os.getcwd()
# print("La mia posizione corrente è:", current_directory)

In [ ]:
# setta la cartella di lavoro
os.chdir(".")

In [ ]:
cancer_drivers = pd.read_csv(r"NCG_cancerdrivers_annotation_supporting_evidence.tsv", sep = "\t")
healthy_drivers = pd.read_csv(r"NCG_healthydrivers_annotation_supporting_evidence.tsv", sep = "\t")

In [ ]:
#uniprot_info = pd.read_csv("uniprot_info.csv")
uniprot_info_2 = pd.read_csv("uniprot_info_2.csv")

## creazione csv drivers (padre), healthy drivers e cancer drivers (figli)

In [ ]:
# aggiungiamo anche gli healthy drivers oltre che i cancer driver
# aggiungiamo anche una colonna che riporta il tipo di driver (cancer o healthy) ad ogni record
cancer_drivers["driver_type"] = "cancer"
# il gene WAS è stato eliminato perchè portava problemi successivamente nella fase di retrieve delle informazioni
cancer_drivers.drop(cancer_drivers[cancer_drivers["symbol"] == "WAS"].index, inplace=True)
healthy_drivers = healthy_drivers[['entrez', 'symbol', 'pubmed_id', 'type', 'organ_system']]
healthy_drivers["driver_type"] = "healthy"
drivers = pd.concat([cancer_drivers, healthy_drivers])

In [ ]:
cancer_drivers

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg,driver_type
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN,cancer
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8881,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",noncoding,NaN,NaN,NaN,NaN,NaN,cancer
8882,100423062,IGLL5,31558468,WGS,Hematologic and lymphatic,blood,burkitt lymphoma,"MutSigCV, HotNet2, Literature-based",coding,NaN,NaN,NaN,NaN,NaN,cancer
8883,100423062,IGLL5,32024818,WGS,Multiple,multiple,pan-cancer_adult,DriverPower,noncoding,NaN,NaN,NaN,NaN,NaN,cancer
8884,100505993,TMEM210,27311963,WGS,Multiple,multiple,pan-cancer_adult,OncodriveFML,noncoding,NaN,NaN,NaN,NaN,NaN,cancer


In [ ]:
healthy_drivers

,entrez,symbol,pubmed_id,type,organ_system,driver_type
0,92,ACVR2A,31645727,WES-Gene Panel,Developmental gastrointestinal,healthy
1,207,AKT1,31187483,Gene Panel,Gynecologic,healthy
2,213,ALB,31645727,WES-Gene Panel,Developmental gastrointestinal,healthy
3,213,ALB,30955891,WES-Gene Panel,Developmental gastrointestinal,healthy
4,338,APOB,30955891,WES-Gene Panel,Developmental gastrointestinal,healthy
...,...,...,...,...,...,...
162,171023,ASXL1,25326804,WES,Hematologic and lymphatic,healthy
163,196528,ARID2,30337457,Gene Panel,Core gastrointestinal,healthy
164,196528,ARID2,33029006,Gene Panel,Skin,healthy
165,196528,ARID2,31996850,WGS,Thoracic,healthy


In [ ]:
drivers

,entrez,symbol,pubmed_id,type,organ_system,primary_site,cancer_type,method,coding_status,cgc_annotation,vogelstein_annotation,saito_annotation,NCG_oncogene,NCG_tsg,driver_type
0,23,ABCF1,31444325,WGS-WES,Hematologic and lymphatic,blood,multiple_myeloma,dNdScv,coding,NaN,NaN,NaN,NaN,NaN,cancer
1,25,ABL1,29625053,Pan-cancer,Multiple,multiple,pan-cancer_adult,PanSoftWare,coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
2,25,ABL1,14993899,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
3,25,ABL1,29489755,Pan-cancer,Multiple,multiple,pan-cancer_paediatric,"GRIN, MutSig",coding,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
4,25,ABL1,23539594,Canonical Cancer Driver,Multiple,multiple,NaN,-,NaN,"oncogene, fusion",Oncogene,NaN,1.0,0.0,cancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,171023,ASXL1,25326804,WES,Hematologic and lymphatic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy
163,196528,ARID2,30337457,Gene Panel,Core gastrointestinal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy
164,196528,ARID2,33029006,Gene Panel,Skin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy
165,196528,ARID2,31996850,WGS,Thoracic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,healthy


In [ ]:
# gene_symbols è una lista che contiene tutti i geni drivers senza avere ripetizioni
gene_symbols = drivers["symbol"].unique()
print(len(gene_symbols))

['ABCF1' 'ABL1' 'ABL2' ... 'PAPPA2' 'GCNA' 'PPARGC1B']


## Creazione csv uniprot_info (riporta l'ac_number su uniprot e poche altre info)

In [ ]:
# creiamo oggetto Uniprot necessario per fare la ricerca sul database
u = UniProt()

In [ ]:
uniprot_info = pd.DataFrame()
i = 0
for gene in gene_symbols:
    info_gene = u.search(query="gene_exact:"+str.strip(gene)+"+AND+organism_id:9606+AND+reviewed:true", frmt="tsv")
    info_gene = pd.read_csv(StringIO(info_gene), sep = "\t")
    info_gene["Gene Name Reference"] = info_gene["Gene Names"].str.split().str[0]
    uniprot_info = pd.concat([uniprot_info, info_gene])
    print(i)
    i +=1

In [ ]:
# AC_uniprot contiene tutti gli accession number
AC_uniprot = list(uniprot_info["Entry"])

## Creazione csv uniprot_info_2 (riporta tutte le info presenti su uniprot)

In [ ]:
uniprot_info_2 = pd.DataFrame()
i =0
for AC in AC_uniprot:
    uniprot_info_2 = pd.concat([uniprot_info_2, u.get_df(AC)])
    print(i)
    i +=1

In [ ]:
uniprot_info_2

,Entry,Entry Name,Gene Names,Gene Names (primary),Gene Names (synonym),Gene Names (ordered locus),Gene Names (ORF),Organism,Organism (ID),Protein names,Proteomes,Taxonomic lineage,Virus hosts,Fragment,Sequence,Length,Mass,Gene encoded by,Alternative products (isoforms),Erroneous gene model prediction,Mass spectrometry,Polymorphism,RNA Editing,Sequence caution,Alternative sequence,Natural variant,Non-adjacent residues,Non-standard residue,Non-terminal residue,Sequence conflict,Sequence uncertainty,Sequence version,Coiled coil,Compositional bias,Domain [CC],Domain [FT],Motif,Protein families,Region,Repeat,Zinc finger,Absorption,Active site,Activity regulation,Binding site,Catalytic activity,Cofactor,DNA binding,EC number,Function [CC],Kinetics,Pathway,pH dependence,Redox potential,Site,Temperature dependence,Gene Ontology (GO),Gene Ontology (biological process),Gene Ontology (molecular function),Gene Ontology (cellular component),Gene Ontology IDs,Interacts with,Subunit structure,Developmental stage,Induction,Tissue specificity,PubMed ID,Date of creation,Date of last modification,Date of last sequence modification,Entry version,3D,Beta strand,Helix,Turn,Subcellular location [CC],Intramembrane,Topological domain,Transmembrane,Annotation,Caution,Comments,Features,Keywords,Keyword ID,Miscellaneous [CC],Protein existence,Tools,Reviewed,UniParc,Allergenic Properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose,Chain,Cross-link,Disulfide bond,Glycosylation,Initiator methionine,Lipidation,Modified residue,Peptide,Post-translational modification,Propeptide,Signal peptide,Transit peptide,PDB
0,Q8NE71,ABCF1_HUMAN,ABCF1 ABC50,ABCF1,ABC50,NaN,NaN,Homo sapiens (Human),9606,ATP-binding cassette sub-family F member 1 (AT...,UP000005640: Chromosome 6,"cellular organisms (no rank), Eukaryota (super...",NaN,NaN,MPKAPKQQPPEPEWIGDGESTSPSDKVVKKGKKDKKIKKTFFEELA...,845,95926,NaN,ALTERNATIVE PRODUCTS: Event=Alternative splic...,NaN,NaN,NaN,NaN,NaN,"VAR_SEQ 226..263; /note=""Missing (in isoform 2...","VARIANT 198; /note=""N -> D (in dbSNP:rs6902544...",NaN,NaN,NaN,"CONFLICT 166; /note=""S -> P (in Ref. 5; AAH344...",NaN,2,NaN,"COMPBIAS 37..63; /note=""Basic and acidic resid...",NaN,"DOMAIN 304..548; /note=""ABC transporter 1""; /e...",NaN,"ABC transporter superfamily, ABCF family, EF3 ...","REGION 1..261; /note=""Disordered""; /evidence=""...",NaN,NaN,NaN,NaN,NaN,"BINDING 336..343; /ligand=""ATP""; /ligand_id=""C...",NaN,NaN,NaN,NaN,FUNCTION: Isoform 2 is required for efficient ...,NaN,NaN,NaN,NaN,NaN,NaN,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,inflammatory response [GO:0006954]; positive r...,ATP binding [GO:0005524]; ATP hydrolysis activ...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,GO:0003723; GO:0005524; GO:0005635; GO:0005654...,NaN,SUBUNIT: Isoform 2 interacts (via N-terminus) ...,NaN,INDUCTION: By TNF in cultured synoviocytes.,TISSUE SPECIFICITY: Ubiquitous. {ECO:0000269|P...,9790762; 14574404; 15489334; 17974005; 1708198...,2004-12-07,2024-03-27,2004-12-07,192,X-ray crystallography (1),"STRAND 304..313; /evidence=""ECO:0007829|PDB:5Z...","HELIX 342..350; /evidence=""ECO:0007829|PDB:5ZX...","TURN 478..481; /evidence=""ECO:0007829|PDB:5ZXD...",SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm {...,NaN,NaN,NaN,5.0,NaN,ALTERNATIVE PRODUCTS (2); FUNCTION (1); INDUCT...,Alternative sequence (1); Beta strand (18); Bi...,3D-structure;Activator;Alternative splicing;AT...,KW-0002; KW-0010; KW-0025; KW-0067; KW-0963; K...,NaN,Evidence at protein level,NaN,reviewed,UPI0000141876,NaN,NaN,NaN,NaN,"MUTAGEN 109; /note=""S->A: Reduces phosphorylat...",NaN,NaN,"CHAIN 1..845; /note=""ATP-binding cassette sub-...",NaN,NaN,NaN,NaN,NaN,"MOD_RES 22; /note=""Phosphoserine""; /evidence=""...",NaN,PTM: Isoform 2 is phosphorylated at phosphoser...,NaN,NaN,NaN,5ZXD;
1,P00519,ABL1_HUMAN,ABL1 ABL JTK7,ABL1,ABL JTK7,NaN,NaN,Homo sapiens (Human),9606,Tyrosine-protein kinase ABL1 (EC 2.7.10.2) (Ab...,UP00

In [ ]:
healthy_drivers.to_csv("healthy_drivers.csv", index=False)
cancer_drivers.to_csv("cancer_drivers.csv", index=False)
drivers.to_csv("drivers.csv", index=False)
uniprot_info.to_csv("uniprot_info.csv", index = False)
uniprot_info_2.to_csv("uniprot_info_2.csv", index = False)

In [ ]:
 # PARTE DEI MOTIVI FUNZIONALI
# la libreria Bio dovrebbe permettere di ottenere i motivi funzionali

## Creazione csv prosite_protein_info_total,  riporta la relazione tra le proteine e siti funzionali
per ogni proteina vengono annotati gli accession_number dei motivi funzionali che presenta

In [ ]:
# queste due proteine davano problemi
uniprot_info_2.drop(1961, inplace=True)
uniprot_info_2.drop(2893, inplace=True)

In [ ]:
i = 0
for sequence in uniprot_info_2["Sequence"]:
    contenuto = ScanProsite.scan(seq = sequence)
    result = contenuto.read()
    xml_data = result.decode('utf-8')
    root = ET.fromstring(xml_data)
    print(i, flush= True)
    uniprot_ac = "".join(uniprot_info_2.loc[uniprot_info_2["Sequence"] == sequence, "Entry"].tolist())
    i+=1
    # Estrazione delle informazioni
    for match in root.findall('.//{urn:expasy:scanprosite}match'):
        sequence_ac = match.find('{urn:expasy:scanprosite}sequence_ac').text
        start = match.find('{urn:expasy:scanprosite}start').text
        stop = match.find('{urn:expasy:scanprosite}stop').text
        signature_ac = match.find('{urn:expasy:scanprosite}signature_ac').text
        with open("prosite_protein_info.csv", 'a', newline='') as file_csv:
            # Crea un oggetto writer
            writer = csv.writer(file_csv)
            nuova_riga = [uniprot_ac, signature_ac, start, stop, sequence_ac]
            writer.writerow(nuova_riga)
            file_csv.close()


In [ ]:
prosite_protein_info = pd.read_csv("prosite_protein_info.csv", names = ["AC_uniprot", "AC_prosite", "start", "stop", "reference_sequence" ])
AC_prosite = prosite_protein_info["AC_prosite"].unique()

In [ ]:
prosite_protein_info = pd.read_csv("prosite_protein_info.csv", names = ["uniprot_ac", "signature_ac", "start", "stop", "sequence_ac"])
prosite_protein_info_2892 = pd.read_csv("prosite_protein_info_2892.csv", names = ["uniprot_ac", "signature_ac", "start", "stop", "sequence_ac"])
prosite_protein_total = pd.concat([prosite_protein_info, prosite_protein_info_2892])

In [ ]:
prosite_protein_total.to_csv("prosite_protein_info_total.csv", index = False)

In [ ]:
prosite_protein_total

,uniprot_ac,signature_ac,start,stop,sequence_ac
0,Q8NE71,PS50893,304,548,USERSEQ1
1,Q8NE71,PS00211,452,466,USERSEQ1
2,Q8NE71,PS50893,625,840,USERSEQ1
3,Q8NE71,PS00211,743,757,USERSEQ1
4,P00519,PS50002,61,121,USERSEQ1
...,...,...,...,...,...
1911,Q9BXP8,PS00092,1055,1061,USERSEQ1
1912,Q9BXP8,PS50923,1462,1521,USERSEQ1
1913,Q9BXP8,PS50923,1523,1592,USERSEQ1
1914,Q9BXP8,PS50923,1593,1648,USERSEQ1


## Creazione csv di prosite_info, contiene tutte le informazioni dei motivi funzionali

In [ ]:
prosite_info = []
handle = open("prosite.dat")
i = 0
# Cicla su ciascun AC_prosite
for ac_prosite in AC_prosite:
    print(i)
    i += 1
    try:
        # Riavvolgi il file e ricrea l'iteratore per garantire che il cursore del file sia all'inizio
        handle.seek(0)
        records = Prosite.parse(handle)

        # Cicla su ciascun record nel file
        for record in records:
            # Verifica se l'accesso Prosite corrente corrisponde a quello cercato
            if record.accession == ac_prosite:
                # Aggiungi gli attributi del record Prosite alla lista prosite_info
                prosite_info.append([record.accession, record.name, record.description, record.pattern])

    except Exception as e:
        print(f"Si è verificato un errore durante l'elaborazione di {ac_prosite}: {e}")

# Scrivi la lista di liste nel file CSV
with open("prosite_info.csv", 'w', newline='') as file_csv:
    writer = csv.writer(file_csv)
    writer.writerows(prosite_info)

# Chiudi il file quando hai finito di utilizzarlo
handle.close()


In [ ]:
prosite_info = pd.read_csv("prosite_info.csv", names = ["accession", "name", "description", "pattern"] )

# Download e stampa delle tabelle.cvs ottenute da SIGNOR (CancerGeneNet) che contengono per ogni proteina in malattie_df il pathway tramite cui danno il fenotipo patologico tipico dei tumori

In [ ]:


proteine = protein_info_2["Gene Names (primary)"]

#definisco una funzione per ottenere la tabella. Il percorso_file_output è il percorso dove sarà salvato il file csv scaricato, di default è dati.csv
def scarica_tabella_tsv_e_salva_csv(url, parametri=None):
    # try mi permette di gestire facilmente eventuali eccezioni nella richiesta http
    try:
        # Effettua la richiesta GET all'URL specificato. I parametri identificano tipo di dati, fenotipo richiesto e l'output
        response = requests.get(url, params=parametri)

        # Verifica lo stato della risposta: quando è uguale a 200 vuol diire che la richesta è andata a buon fine
        if response.status_code == 200:
            # Salva la risposta in un file CSV
            with open("signor_info.csv", "a") as file:
                risposta_tsv = response.text
                df_tsv = pd.read_csv(StringIO(risposta_tsv), sep='\t')
                df_tsv.to_csv(file, index=False, header=False)
            print(f"Tabella TSV scaricata e salvata come file CSV  con successo!")
        else:
            # Se la risposta non è OK, stampa un messaggio di errore
            print(f"Errore nella richiesta. Codice di stato: {response.status_code}")
    except Exception as e:
        # Gestione generica delle eccezioni
        print(f"Si è verificato un errore: {e}")



#Dati forniti: URL da cui scaricare e parametri. I parametri sottostanti sono sempre gli stessi, in particolare mettendo pathway come vuoto vengono scaricati tutti i pathway presenti nel database
url_api = "https://signor.uniroma2.it/CancerGeneNet/getData.php"
parametri_base = {
    "type": "shortestPath",
    "phenotype": "",
    "output": "summary"
}




#implementazione del codice: qui si scaricano e visualizzano le tabelle iterando per ogni proteina che abbiamo nel nostro dataframe malattie_df
for proteina in proteine:
    #aggiorna i parametri della richiesta con la proteina corrente
    parametri = {**parametri_base, "proteins": proteina}
    #chiamiamo la funzione per scaricare la tabella csv
    scarica_tabella_tsv_e_salva_csv(url_api, parametri)


